In [1]:
import sqlite3

import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
__file__ = %pwd
DATA = Path(__file__).parent / "data"
ELA = DATA / "ela"
DATABASE = DATA / "databases" / "exploration.db"

In [3]:
ela24 = pd.read_excel(ELA / "eladownload2024.xlsx")
ela23 = pd.read_excel(ELA / "eladownload2023.xlsx")
ela22 = pd.read_excel(ELA / "eladownload2022.xlsx")
ela19 = pd.read_excel(ELA / "eladownload2019.xlsx")
ela18 = pd.read_excel(ELA / "eladownload2018.xlsx")

ela24 = ela24[(ela24["charter_flag"] != "Y") & (ela24["coe_flag"] != "Y") &
                (ela24["dass_flag"] != "Y") & (ela24["cds"] != 0) & (ela24["cds"] != 1)]
ela23 = ela23[(ela23["charter_flag"] != "Y") & (ela23["coe_flag"] != "Y") &
                (ela23["dass_flag"] != "Y") & (ela23["cds"] != 0) & (ela23["cds"] != 1)]
ela22 = ela22[(ela22["charter_flag"] != "Y") & (ela22["coe_flag"] != "Y") &
                (ela22["dass_flag"] != "Y") & (ela22["cds"] != 0) & (ela22["cds"] != 1)]
ela19 = ela19[(ela19["charter_flag"] != "Y") & (ela19["coe_flag"] != "Y") &
                (ela19["dass_flag"] != "Y") & (ela19["cds"] != 0) & (ela19["cds"] != 1)]
ela18 = ela18[(ela18["charter_flag"] != "Y") & (ela18["coe_flag"] != "Y") &
                (ela18["dass_flag"] != "Y") & (ela18["cds"] != 0) & (ela18["cds"] != 1)]

ela17 = ela18[["cds", "priordenom", "priorstatus", "studentgroup"]].copy()

In [4]:
ela24["Year"] = "2023-24"
ela23["Year"] = "2022-23"
ela22["Year"] = "2021-22"
ela19["Year"] = "2018-19"
ela18["Year"] = "2017-18"
ela17["Year"] = "2016-17"

In [5]:
ela24 = ela24[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
ela23 = ela23[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
ela22 = ela22[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
ela19 = ela19[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]
ela18 = ela18[["cds", "currdenom", "currstatus", "studentgroup", "Year"]]

ela24.columns = ela23.columns = ela22.columns = ["cdsCode", "currdenom", "currstatus", "studentgroup", "Year"]
ela19.columns = ela18.columns = ela17.columns = ["cdsCode", "currdenom", "currstatus", "studentgroup", "Year"]

In [6]:
def get_countyCode(cds_code: int) -> int:
    county_digits = 1e12
    return int(cds_code / county_digits)

def get_districtCode(cds_code: int) -> int:
    district_digits = 1e7
    county_digits = 1e12
    return int((cds_code % county_digits) / district_digits)

def get_schoolCode(cds_code: int) -> int:
    district_digits = 1e7
    return int(cds_code % district_digits)

def cds_code_split(cds_code: int) -> pd.Series:
    return pd.Series([get_countyCode(cds_code), get_districtCode(cds_code), get_schoolCode(cds_code)])

In [7]:
ela24[["CountyCode", "DistrictCode", "SchoolCode"]] = ela24.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
ela23[["CountyCode", "DistrictCode", "SchoolCode"]] = ela23.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
ela22[["CountyCode", "DistrictCode", "SchoolCode"]] = ela22.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
ela19[["CountyCode", "DistrictCode", "SchoolCode"]] = ela19.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
ela18[["CountyCode", "DistrictCode", "SchoolCode"]] = ela18.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)
ela17[["CountyCode", "DistrictCode", "SchoolCode"]] = ela17.apply(lambda row: cds_code_split(int(row["cdsCode"])), axis=1)

In [8]:
ela = pd.concat([ela24, ela23, ela22, ela19, ela18, ela17]).drop_duplicates()

In [9]:
ela

,cdsCode,currdenom,currstatus,studentgroup,Year,CountyCode,DistrictCode,SchoolCode
248,1316090131755,1.0,NaN,AA,2023-24,1,31609,131755
249,1316090131755,19.0,-116.2,ALL,2023-24,1,31609,131755
250,1316090131755,3.0,NaN,AS,2023-24,1,31609,131755
251,1316090131755,5.0,NaN,CAA,2023-24,1,31609,131755
252,1316090131755,8.0,NaN,EL,2023-24,1,31609,131755
...,...,...,...,...,...,...,...,...
148833,58727695838305,0.0,NaN,PI,2016-17,58,72769,5838305
148834,58727695838305,3.0,NaN,RFP,2016-17,58,72769,5838305
148835,58727695838305,54.0,-6.3,SED,2016-17,58,72769,5838305
148836,58727695838305,9.0,NaN,SWD,2016-17,58,72769,5838305


In [10]:
con = sqlite3.connect(DATABASE)
ela.to_sql("ELA", con, index=False)
con.close()